# Load data from CSV, parse it appropriately

This script loads the data from a CSV file and parses the data for concepts required for the GOSISS project. The script outputs the `anzics-gosiss-data.csv` file for later use.

Requirements:

* `anzics_2008_2015_deidentified.csv`

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from collections import OrderedDict

%matplotlib inline

## Create the mapping from ANZICS variables to GOSSIS variables

In [ ]:
# encounter id is a concatenation of of:
#   - siteid, patientid, admepisode (incrementing integer for each ICU stay)
encFcn = lambda x: 'anzics_' + x['site_id'].astype(str) + '_' + \
(x['patientid']).astype(str) + '_' + (x['admepisode']).astype(str)

dict_hosptype = {
    1:'Rural/Regional',
    2:'Metropolitan',
    3:'Tertiary/Teaching',
    4:'Private'}

dict_caretype = {1:'ICU',2:'HDU'}

dict_hospadmit = {1:'Home',
2:'Other Acute Hospital',
3:'Chronic Care Hospital (including nursing homes)',
4:'Other hospital ICU'}

dict_hospoutcome = {
    2:'Died in Hospital',
    3:'Discharged home',
    4:'Transferred to Chronic Care/Rehabilitation Hospital',
    5:'Transferred to Other Hospital ICU',
    6:'Transferred to Other Acute Care Hospital'}

dict_icuoutcome = {
2:'Died in ICU',
3:'Survived ICU',
5:'Transferred to another ICU',
6:'Transferred to another hospital'
}

field_map = OrderedDict([['encounterId', encFcn]
, ['patientId', 'patientid']
#, [None, 'countrycode']
, ['country', 'country']
, ['hospitalId', 'site_id']
, ['teaching_hospital', lambda x: x['cicmlevel'] == 3]
, ['hospital_bed_size', None]
, ['hospital_type', lambda x: dict_hosptype[x['hospitalclassificationid']]]
#, [None, 'locationcode']
#, [None, 'jurisdictionid']
#, [None, 'jurisdictionname']
#, [None, 'publicprivateid']
#, [None, 'publicprivate']
#, [None, 'cicmlevel']
#, [None, 'cicmlevelid']
, ['icuId', 'site_id'] # anzics doesn't identify individual ICUs
, ['icu_type', None]
, ['icu_stay_type', lambda x: dict_caretpye[x['caretype']]]
, ['age', 'age']
, ['gender', 'sex']
, ['weight', 'weight']
, ['height', 'height']
, ['bmi', None]
, ['ethnicity', 'indigenous']
#, [None, 'postcode']
, ['pregnant', 'preg_stat']
, ['smoking_status', 'smokingstatus']
#, [None, 'smokingintensity']
#, [None, 'icuadmissioncount']
#, [None, 'hospitaladmissioncount']
#, [None, 'readmissioncount']
#, [None, 'prior_icu_ad_dtm']
#, [None, 'prior_icu_ds_dtm']
#, [None, 'hosp_ad_dtm']
#, [None, 'hosp_ds_dtm']
, ['hospital_admit_source', lambda x: dict_hospadmit[x['hosp_srce']]]
, ['hospital_disch_location', lambda x: dict_hospoutcome[x['hosp_outcm']]]
, ['hospital_los_days', 'hosp_hrs']
, ['hospital_death', 'died_hosp']
, ['icu_admit_source', 'icu_srce']
, ['icu_admit_type', None]
, ['icu_disch_location', lambda x: dict_icuoutcome[x['icu_outcm']]]
, ['pre_icu_los_days', 'pre_icu_hrs']
, ['icu_los_days', 'icu_hrs']
, ['icu_death', 'died_icu']
#, [None, 'admepisode']
#, [None, 'icuadmitfinyr']
#, [None, 'icuadmityyyymm']
#, [None, 'icuadmityyyy']
#, [None, 'icu_ad_dtm']
#, [None, 'icu_ds_dtm']
, ['elective_surgery', lambda x: x['elect']==2]
, ['readmission_status', lambda x: x['readmitted']==2]
#, [None, 'icu_ds_dec_dtm']
#, [None, 'discharge_delay_hrs']
#, [None, 'readmission_lag_hrs']
#, [None, 'icu_outcm']
#, [None, 'emg_rsp_adm']
, ['GIBleed_1h', None]
, ['SNCMass_1h', None]
, ['Aminas_1h', None]
, ['Arrhythmia_1h', None]
, ['AKI_1h', None]
, ['Arrhythmia_1d', None]
, ['CPA_1d', None]
, ['AKI_1d', None]
, ['GIBleed_1d', None]
, ['SNCMass_1d', None]
, ['Neutropenia_1d', None]
#, [None, 'cardarrest']
#, [None, 'resparrest']
#, [None, 'cabg_graft']
#, [None, 'cabg_redo']
#, [None, 'thromb_therapy']
#, [None, 'chr_resp']
#, [None, 'chr_liv']
#, [None, 'arf']
#, [None, 'thrombpro']
#, [None, 'iddm']
#, [None, 'immundis']
#, [None, 'immunrx']
#, [None, 'chr_cvs']
#, [None, 'cirrhos']
#, [None, 'hepfail']
#, [None, 'lymphoma']
#, [None, 'leukaem']
#, [None, 'metast']
#, [None, 'immunsup']
#, [None, 'aids']
#, [None, 'ap2diag']
#, [None, 'ap3diag']
#, [None, 'ap3_subcode']
, ['d1_heartrate_min', 'hrlo']
, ['d1_heartrate_max', 'hrhi']
, ['d1_resprate_min', 'rrlo']
, ['d1_resprate_max', 'rrhi']
, ['d1_spo2_min', None]
, ['d1_spo2_max', None]
, ['d1_temp_min', 'templo']
, ['d1_temp_max', 'temphi']
, ['d1_sysbp_invasive_min', None]
, ['d1_sysbp_invasive_max', None]
, ['d1_diasbp_invasive_min', None]
, ['d1_diasbp_invasive_max', None]
, ['d1_mbp_invasive_min', None]
, ['d1_mbp_invasive_max', None]
, ['d1_sysbp_noninvasive_min', None]
, ['d1_sysbp_noninvasive_max', None]
, ['d1_diasbp_noninvasive_min', None]
, ['d1_diasbp_noninvasive_max', None]
, ['d1_mbp_noninvasive_min', None]
, ['d1_mbp_noninvasive_max', None]
, ['d1_sysbp_min', 'systoliclo']
, ['d1_sysbp_max', 'systolichi']
, ['d1_diasbp_min', 'diastoliclo']
, ['d1_diasbp_max', 'diastolichi']
, ['d1_mbp_min', 'maplo']
, ['d1_mbp_max', 'maphi']
, ['d1_pasys_invasive_min', None]
, ['d1_pasys_invasive_max', None]
, ['d1_padias_invasive_min', None]
, ['d1_padias_invasive_max', None]
, ['d1_pamean_invasive_min', None]
, ['d1_pamean_invasive_max', None]
, ['h1_heartrate_min', None]
, ['h1_heartrate_max', None]
, ['h1_resprate_min', None]
, ['h1_resprate_max', None]
, ['h1_spo2_min', None]
, ['h1_spo2_max', None]
, ['h1_temp_min', None]
, ['h1_temp_max', None]
, ['h1_sysbp_invasive_min', None]
, ['h1_sysbp_invasive_max', None]
, ['h1_diasbp_invasive_min', None]
, ['h1_diasbp_invasive_max', None]
, ['h1_mbp_invasive_min', None]
, ['h1_mbp_invasive_max', None]
, ['h1_sysbp_noninvasive_min', None]
, ['h1_sysbp_noninvasive_max', None]
, ['h1_diasbp_noninvasive_min', None]
, ['h1_diasbp_noninvasive_max', None]
, ['h1_mbp_noninvasive_min', None]
, ['h1_mbp_noninvasive_max', None]
, ['h1_sysbp_min', None]
, ['h1_sysbp_max', None]
, ['h1_diasbp_min', None]
, ['h1_diasbp_max', None]
, ['h1_mbp_min', None]
, ['h1_mbp_max', None]
, ['h1_pasys_invasive_min', None]
, ['h1_pasys_invasive_max', None]
, ['h1_padias_invasive_min', None]
, ['h1_padias_invasive_max', None]
, ['h1_pamean_invasive_min', None]
, ['h1_pamean_invasive_max', None]
, ['d1_albumin_min', None]
, ['d1_albumin_max', None]
, ['d1_bilirubin_min', None]
, ['d1_bilirubin_max', None]
, ['d1_bun_min', None]
, ['d1_bun_max', None]
, ['d1_calcium_min', None]
, ['d1_calcium_max', None]
, ['d1_creatinine_min', 'creatlo']
, ['d1_creatinine_max', 'creathi']
, ['d1_glucose_min', 'gluclo']
, ['d1_glucose_max', 'gluchi']
, ['d1_inr_min', None]
, ['d1_inr_max', None]
, ['d1_hco3_min', 'hco3lo']
, ['d1_hco3_max', 'hco3hi']
, ['d1_hematocrit_min', 'hctlo']
, ['d1_hematocrit_max', 'hcthi']
, ['d1_hemaglobin_min', 'hmgnlo']
, ['d1_hemaglobin_max', 'hmgnhi']
, ['d1_lactate_min', None]
, ['d1_lactate_max', None]
, ['d1_platelets_min', 'platlo']
, ['d1_platelets_max', 'plathi']
, ['d1_potassium_min', 'klo']
, ['d1_potassium_max', 'khi']
, ['d1_sodium_min', 'nalo']
, ['d1_sodium_max', 'nahi']
, ['d1_wbc_min', 'wcclo']
, ['d1_wbc_max', 'wcchi']
, ['d1_arterial_ph_min', None]
, ['d1_arterial_ph_max', None]
, ['d1_arterial_po2_min', None]
, ['d1_arterial_po2_max', None]
, ['d1_arterial_pco2_min', None]
, ['d1_arterial_pco2_max', None]
, ['d1_PaO2FiO2Ratio_min', None]
, ['h1_albumin_min', None]
, ['h1_albumin_max', None]
, ['h1_bilirubin_min', None]
, ['h1_bilirubin_max', None]
, ['h1_bun_min', None]
, ['h1_bun_max', None]
, ['h1_calcium_min', None]
, ['h1_calcium_max', None]
, ['h1_creatinine_min', None]
, ['h1_creatinine_max', None]
, ['h1_glucose_min', None]
, ['h1_glucose_max', None]
, ['h1_inr_min', None]
, ['h1_inr_max', None]
, ['h1_hco3_min', None]
, ['h1_hco3_max', None]
, ['h1_hematocrit_min', None]
, ['h1_hematocrit_max', None]
, ['h1_hemaglobin_min', None]
, ['h1_hemaglobin_max', None]
, ['h1_lactate_min', None]
, ['h1_lactate_max', None]
, ['h1_platelets_min', None]
, ['h1_platelets_max', None]
, ['h1_potassium_min', None]
, ['h1_potassium_max', None]
, ['h1_sodium_min', None]
, ['h1_sodium_max', None]
, ['h1_wbc_min', None]
, ['h1_wbc_max', None]
, ['h1_arterial_ph_min', None]
, ['h1_arterial_ph_max', None]
, ['h1_arterial_po2_min', None]
, ['h1_arterial_po2_max', None]
, ['h1_arterial_pco2_min', None]
, ['h1_arterial_pco2_max', None]
, ['h1_PaO2FiO2Ratio_min', None]
, ['highpao2fio2', None]
, ['lowpao2fio2', None]
, ['d1_gcs_verbal', 'gcsverb']
, ['d1_gcs_motor', 'gcsmotor']
, ['d1_gcs_eyes', 'gcseye']
#, [None, 'TREAT_LMT']
, ['h1_gcs_verbal', None]
, ['h1_gcs_motor', None]
, ['h1_gcs_eyes', None]
, ['h1_gcs_total_min', None]
, ['albumin_apache', 'albumin']
#, [None, 'ap3co2o']
#, [None, 'ap3co2p']
#, [None, 'ap3fio']
#, [None, 'ap3ph']
#, [None, 'ap3po2']
, ['bilirubin_apache', 'bili']
, ['creatinine_apache', 'creat']
, ['fio2_apache', 'fio2']
, ['glucose_apache', 'glucose']
, ['bicarbonate_apache', 'hco3']
, ['hematocrit_apache', 'hct']
, ['heart_rate_apache', 'hr']
, ['potassium_apache', 'k']
, ['map_apache', 'map']
, ['sodium_apache', 'na']
, ['paco2_apache', 'paco2']
, ['pao2_apache', 'pao2']
, ['ph_apache', 'ph']
, ['resprate_apache', 'rr']
, ['temp_apache', 'temp']
, ['bun_apache', 'urea']
, ['urineoutput_apache', 'urineop']
, ['wbc_apache', 'wcc']
, ['gcs_eyes_apache', 'gcseye']
, ['gcs_motor_apache', 'gcsmotor']
, ['gcs_verbal_apache', 'gcsverb']
, ['gcs_unable_apache', None]
#, [None, 'intubated']
#, [None, 'ventilated']
#, [None, 'chr_ren'] # APACHE-II chronic renal failure == use of dialysis
#, [None, 'apache3riskofdeath']
#, [None, 'apache3score']
#, [None, 'apache3isincluded']
#, [None, 'apache3issmr']
, ['APACHE_IVa_score', None]
, ['APACHE_IVa_icu_death_prob', None]
, ['APACHE_IVa_hospital_death_prob', None]
#, [None, 'anzrodriskofdeath']
#, [None, 'anzrodisincluded']
#, [None, 'anzrodissmr']
#, [None, 'agescore']
#, [None, 'albuminscore']
#, [None, 'bilirubinscore']
#, [None, 'creatininescore']
#, [None, 'glucosescore']
#, [None, 'haematocritscore']
#, [None, 'heartratescore']
#, [None, 'meanarterialpressurescore']
#, [None, 'sodiumscore']
#, [None, 'neurologicalscore']
#, [None, 'oxygenationscore']
#, [None, 'phscore']
#, [None, 'respiratoryratescore']
#, [None, 'temperaturescore']
#, [None, 'ureascore']
#, [None, 'urineoutputscore']
#, [None, 'whitecellcountscore']
#, [None, 'riskmodelversionid']
])

## Load in the data

In [ ]:
df = pd.read_csv('anzics_mini.csv',header=0,sep=',')
print([c for c in df.columns])

In [ ]:
df = pd.read_csv('anzics_2008_2015_deidentified.csv',header=0,sep=',')
#df = pd.read_csv('anzics_mini.csv',header=0,sep=',')

# convert columns to lower case
df.columns = [c.lower() for c in df.columns]

# the first column, site_id, has some weird text that we strip
c = df.columns[0]
if 'site_id' in c:
    df.columns = ['site_id' if (i==0) & ('site_id' in c) else c for i, c in enumerate(df.columns)]
else:
    print('WARNING: The first column may have changed. Check it for bad characters e.g. \xef\xbb\xbf')

In [ ]:
df['ap3co2p'].value_counts() #/ (577001+426575.0)

In [ ]:
df.loc[df['patientid']=='8791',:].sort_values('icu_ad_dtm').T

In [ ]:
df.loc[df['readmitted']==1,:].head().T

In [ ]:
# combine site/patientid/admission episode (icu encounter #) to form a unique encounter id
encFcn = lambda x: 'anzics_' + x['site_id'].astype(str) + '_' + \
(x['patientid']).astype(str) + '_' + (x['admepisode']).astype(str)

field_map = OrderedDict([['encounterId', encFcn]
, ['patientId', 'patientid'] ])

In [ ]:
df_new = pd.DataFrame()
for c in field_map:
    if field_map[c] is None:
        df_new[c] = None
    elif type(field_map[c]) == str:
        if field_map[c].lower() in df.columns.values:
            # data exists, copy it over
            df_new[c] = df[field_map[c]]
        else:
            print('WARNING: {} equivalent not found in ANZICS data! (Looked for "{}".)'.format(c, field_map[c].lower()))
    else:
        # call the function
        df_new[c] = field_map[c](df)

In [ ]:
df_new

# 3 - Output the data to a csv file

In [ ]:
df_new.to_csv('anzics-gosiss-data.csv',index=False)

# Appendix

Same as the above dictionary, but reversed.

```python
field_map = OrderedDict([['patientid', 'encounterId']
#, [TODO, patientId]
, ['countrycode', None]
, ['country', 'country']
, ['site_id', 'hospitalId']
#, [TODO, teaching_hospital]
#, [TODO, hospital_bed_size]
, ['hospitalclassification', 'hospital_type']
, ['hospitalclassificationid', None]
, ['locationcode', None]
, ['jurisdictionid', None]
, ['jurisdictionname', None]
, ['publicprivateid', None]
, ['publicprivate', None]
, ['cicmlevel', None]
, ['cicmlevelid', None]
#, [TODO, icuId]
, ['careunit', 'icu_type']
, ['caretype', 'icu_stay_type']
, ['age', 'age']
, ['sex', 'gender']
, ['weight', 'weight']
, ['height', 'height']
#, [TODO, bmi]
, ['indigenous', 'ethnicity']
, ['postcode', None]
, ['preg_stat', 'pregnant']
, ['smokingstatus', 'smoking_status']
, ['smokingintensity', None]
, ['icuadmissioncount', None]
, ['hospitaladmissioncount', None]
, ['readmissioncount', None]
, ['prior_icu_ad_dtm', None]
, ['prior_icu_ds_dtm', None]
, ['hosp_ad_dtm', None]
, ['hosp_ds_dtm', None]
, ['hosp_srce', 'hospital_admit_source']
#, [TODO, hospital_disch_location]
, ['hosp_hrs', 'hospital_los_days']
, ['died_hosp', 'hospital_death']
, ['hosp_outcm', None]
, ['icu_srce', 'icu_admit_source']
#, [TODO, icu_admit_type]
#, [TODO, icu_disch_location]
, ['pre_icu_hrs', 'pre_icu_los_days']
, ['icu_hrs', 'icu_los_days']
, ['died_icu', 'icu_death']
, ['admepisode', None]
, ['icuadmitfinyr', None]
, ['icuadmityyyymm', None]
, ['icuadmityyyy', None]
, ['icu_ad_dtm', None]
, ['icu_ds_dtm', None]
, ['elect', 'elective_surgery']
, ['readmitted', 'readmission_status']
, ['icu_ds_dec_dtm', None]
, ['discharge_delay_hrs', None]
, ['readmission_lag_hrs', None]
, ['icu_outcm', None]
, ['emg_rsp_adm', None]
#, [TODO, GIBleed_1h]
#, [TODO, SNCMass_1h]
#, [TODO, Aminas_1h]
#, [TODO, Arrhythmia_1h]
#, [TODO, AKI_1h]
#, [TODO, Arrhythmia_1d]
#, [TODO, CPA_1d]
#, [TODO, AKI_1d]
#, [TODO, GIBleed_1d]
#, [TODO, SNCMass_1d]
#, [TODO, Neutropenia_1d]
, ['cardarrest', None]
, ['resparrest', None]
, ['intubated', None]
, ['ventilated', None]
, ['cabg_graft', None]
, ['cabg_redo', None]
, ['thromb_therapy', None]
, ['chr_resp', None]
, ['chr_liv', None]
, ['arf', None]
, ['thrombpro', None]
, ['iddm', None]
, ['immundis', None]
, ['immunrx', None]
, ['chr_cvs', None]
, ['chr_ren', None]
, ['cirrhos', None]
, ['hepfail', None]
, ['lymphoma', None]
, ['leukaem', None]
, ['metast', None]
, ['immunsup', None]
, ['aids', None]
, ['ap2diag', None]
, ['ap3diag', None]
, ['ap3_subcode', None]
, ['hrlo', 'd1_heartrate_min']
, ['hrhi', 'd1_heartrate_max']
, ['rrlo', 'd1_resprate_min']
, ['rrhi', 'd1_resprate_max']
#, [TODO, d1_spo2_min]
#, [TODO, d1_spo2_max]
, ['templo', 'd1_temp_min']
, ['temphi', 'd1_temp_max']
#, [TODO, d1_sysbp_invasive_min]
#, [TODO, d1_sysbp_invasive_max]
#, [TODO, d1_diasbp_invasive_min]
#, [TODO, d1_diasbp_invasive_max]
#, [TODO, d1_mbp_invasive_min]
#, [TODO, d1_mbp_invasive_max]
#, [TODO, d1_sysbp_noninvasive_min]
#, [TODO, d1_sysbp_noninvasive_max]
#, [TODO, d1_diasbp_noninvasive_min]
#, [TODO, d1_diasbp_noninvasive_max]
#, [TODO, d1_mbp_noninvasive_min]
#, [TODO, d1_mbp_noninvasive_max]
, ['systoliclo', 'd1_sysbp_min']
, ['systolichi', 'd1_sysbp_max']
, ['diastoliclo', 'd1_diasbp_min']
, ['diastolichi', 'd1_diasbp_max']
, ['maplo', 'd1_mbp_min']
, ['maphi', 'd1_mbp_max']
#, [TODO, d1_pasys_invasive_min]
#, [TODO, d1_pasys_invasive_max]
#, [TODO, d1_padias_invasive_min]
#, [TODO, d1_padias_invasive_max]
#, [TODO, d1_pamean_invasive_min]
#, [TODO, d1_pamean_invasive_max]
#, [TODO, h1_heartrate_min]
#, [TODO, h1_heartrate_max]
#, [TODO, h1_resprate_min]
#, [TODO, h1_resprate_max]
#, [TODO, h1_spo2_min]
#, [TODO, h1_spo2_max]
#, [TODO, h1_temp_min]
#, [TODO, h1_temp_max]
#, [TODO, h1_sysbp_invasive_min]
#, [TODO, h1_sysbp_invasive_max]
#, [TODO, h1_diasbp_invasive_min]
#, [TODO, h1_diasbp_invasive_max]
#, [TODO, h1_mbp_invasive_min]
#, [TODO, h1_mbp_invasive_max]
#, [TODO, h1_sysbp_noninvasive_min]
#, [TODO, h1_sysbp_noninvasive_max]
#, [TODO, h1_diasbp_noninvasive_min]
#, [TODO, h1_diasbp_noninvasive_max]
#, [TODO, h1_mbp_noninvasive_min]
#, [TODO, h1_mbp_noninvasive_max]
#, [TODO, h1_sysbp_min]
#, [TODO, h1_sysbp_max]
#, [TODO, h1_diasbp_min]
#, [TODO, h1_diasbp_max]
#, [TODO, h1_mbp_min]
#, [TODO, h1_mbp_max]
#, [TODO, h1_pasys_invasive_min]
#, [TODO, h1_pasys_invasive_max]
#, [TODO, h1_padias_invasive_min]
#, [TODO, h1_padias_invasive_max]
#, [TODO, h1_pamean_invasive_min]
#, [TODO, h1_pamean_invasive_max]
#, [TODO, d1_albumin_min]
#, [TODO, d1_albumin_max]
#, [TODO, d1_bilirubin_min]
#, [TODO, d1_bilirubin_max]
#, [TODO, d1_bun_min]
#, [TODO, d1_bun_max]
#, [TODO, d1_calcium_min]
#, [TODO, d1_calcium_max]
, ['creatlo', 'd1_creatinine_min']
, ['creathi', 'd1_creatinine_max']
, ['gluclo', 'd1_glucose_min']
, ['gluchi', 'd1_glucose_max']
#, [TODO, d1_inr_min]
#, [TODO, d1_inr_max]
, ['hco3lo', 'd1_hco3_min']
, ['hco3hi', 'd1_hco3_max']
, ['hctlo', 'd1_hematocrit_min']
, ['hcthi', 'd1_hematocrit_max']
, ['hmgnlo', 'd1_hemaglobin_min']
, ['hmgnhi', 'd1_hemaglobin_max']
#, [TODO, d1_lactate_min]
#, [TODO, d1_lactate_max]
, ['platlo', 'd1_platelets_min']
, ['plathi', 'd1_platelets_max']
, ['klo', 'd1_potassium_min']
, ['khi', 'd1_potassium_max']
, ['nalo', 'd1_sodium_min']
, ['nahi', 'd1_sodium_max']
, ['wcclo', 'd1_wbc_min']
, ['wcchi', 'd1_wbc_max']
#, [TODO, d1_arterial_ph_min]
#, [TODO, d1_arterial_ph_max]
#, [TODO, d1_arterial_po2_min]
#, [TODO, d1_arterial_po2_max]
#, [TODO, d1_arterial_pco2_min]
#, [TODO, d1_arterial_pco2_max]
#, [TODO, d1_PaO2FiO2Ratio_min]
#, [TODO, h1_albumin_min]
#, [TODO, h1_albumin_max]
#, [TODO, h1_bilirubin_min]
#, [TODO, h1_bilirubin_max]
#, [TODO, h1_bun_min]
#, [TODO, h1_bun_max]
#, [TODO, h1_calcium_min]
#, [TODO, h1_calcium_max]
#, [TODO, h1_creatinine_min]
#, [TODO, h1_creatinine_max]
#, [TODO, h1_glucose_min]
#, [TODO, h1_glucose_max]
#, [TODO, h1_inr_min]
#, [TODO, h1_inr_max]
#, [TODO, h1_hco3_min]
#, [TODO, h1_hco3_max]
#, [TODO, h1_hematocrit_min]
#, [TODO, h1_hematocrit_max]
#, [TODO, h1_hemaglobin_min]
#, [TODO, h1_hemaglobin_max]
#, [TODO, h1_lactate_min]
#, [TODO, h1_lactate_max]
#, [TODO, h1_platelets_min]
#, [TODO, h1_platelets_max]
#, [TODO, h1_potassium_min]
#, [TODO, h1_potassium_max]
#, [TODO, h1_sodium_min]
#, [TODO, h1_sodium_max]
#, [TODO, h1_wbc_min]
#, [TODO, h1_wbc_max]
#, [TODO, h1_arterial_ph_min]
#, [TODO, h1_arterial_ph_max]
#, [TODO, h1_arterial_po2_min]
#, [TODO, h1_arterial_po2_max]
#, [TODO, h1_arterial_pco2_min]
#, [TODO, h1_arterial_pco2_max]
#, [TODO, h1_PaO2FiO2Ratio_min]
#, [TODO, highpao2fio2]
#, [TODO, lowpao2fio2]
, ['gcsverb', 'd1_gcs_verbal']
, ['gcsmotor', 'd1_gcs_motor']
, ['gcseye', 'd1_gcs_eyes']
, ['TREAT_LMT', None]
#, [TODO, h1_gcs_verbal]
#, [TODO, h1_gcs_motor]
#, [TODO, h1_gcs_eyes]
#, [TODO, h1_gcs_total_min]
, ['albumin', 'albumin_apache']
, ['ap3co2o', None]
, ['ap3co2p', None]
, ['ap3fio', None]
, ['ap3ph', None]
, ['ap3po2', None]
, ['bili', 'bilirubin_apache']
, ['creat', 'creatinine_apache']
, ['fio2', 'fio2_apache']
, ['glucose', 'glucose_apache']
, ['hco3', 'bicarbonate_apache']
, ['hct', 'hematocrit_apache']
, ['hr', 'heart_rate_apache']
, ['k', 'potassium_apache']
, ['map', 'map_apache']
, ['na', 'sodium_apache']
, ['paco2', 'paco2_apache']
, ['pao2', 'pao2_apache']
, ['ph', 'ph_apache']
, ['rr', 'resprate_apache']
, ['temp', 'temp_apache']
, ['urea', 'bun_apache']
, ['urineop', 'urineoutput_apache']
, ['wcc', 'wbc_apache']
#, [TODO, gcs_eyes_apache]
#, [TODO, gcs_motor_apache]
#, [TODO, gcs_verbal_apache]
#, [TODO, gcs_unable_apache]
, ['apache3riskofdeath', None]
, ['apache3score', None]
, ['apache3isincluded', None]
, ['apache3issmr', None]
#, [TODO, APACHE_IVa_score]
#, [TODO, APACHE_IVa_icu_death_prob]
#, [TODO, APACHE_IVa_hospital_death_prob]
, ['anzrodriskofdeath', None]
, ['anzrodisincluded', None]
, ['anzrodissmr', None]
, ['agescore', None]
, ['albuminscore', None]
, ['bilirubinscore', None]
, ['creatininescore', None]
, ['glucosescore', None]
, ['haematocritscore', None]
, ['heartratescore', None]
, ['meanarterialpressurescore', None]
, ['sodiumscore', None]
, ['neurologicalscore', None]
, ['oxygenationscore', None]
, ['phscore', None]
, ['respiratoryratescore', None]
, ['temperaturescore', None]
, ['ureascore', None]
, ['urineoutputscore', None]
, ['whitecellcountscore', None]
, ['riskmodelversionid', None]
])
```